In [46]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [47]:
train_path= pd.read_csv('IA1_train.csv')
val_path= pd.read_csv('IA1_dev.csv')
val_path

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,price
0,3211200460,8/6/2014,4,1.00,1520,9800,1.5,0,0,4,...,1520,0,1971,0,98034,47.7303,-122.236,1540,7700,3.8900
1,4124000320,3/16/2015,3,2.25,1800,15903,1.0,0,0,3,...,1340,460,1986,0,98038,47.3813,-122.043,2000,15233,3.3562
2,7129302800,12/12/2014,3,1.50,1780,5000,1.0,0,4,4,...,1030,750,1958,0,98118,47.5168,-122.256,1780,7500,4.2000
3,1392800035,6/18/2014,2,1.00,1240,6400,1.0,0,1,4,...,1060,180,1938,0,98126,47.5493,-122.377,1240,6400,5.5900
4,2154900040,10/30/2014,3,2.25,2190,8834,1.0,0,0,3,...,1390,800,1987,0,98001,47.2633,-122.244,1490,8766,1.9425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,5132000140,1/20/2015,6,1.00,1370,5080,1.5,0,0,3,...,1120,250,1931,0,98106,47.5238,-122.350,1020,5080,4.1500
1996,6624010170,5/8/2014,3,1.75,1390,7399,1.0,0,0,4,...,1390,0,1975,0,98031,47.4183,-122.182,1460,7800,2.4600
1997,1853080840,2/11/2015,5,3.50,3700,7055,2.0,0,0,3,...,3700,0,2014,0,98074,47.5929,-122.057,3170,6527,8.8995
1998,2767601311,10/24/2014,3,2.50,1260,1102,3.0,0,0,3,...,1260,0,2007,0,98107,47.6750,-122.387,1320,2500,4.4500


In [48]:
#remove the ID column from both training and validation data
train_without_id = train_path.drop(columns=['id'])
train_without_id.dtypes
test_without_id = val_path.drop(columns=['id'])
test_without_id.dtypes

date              object
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
price            float64
dtype: object

In [49]:
#change date into 3 numerical features day, month and year
train_without_id['date']=pd.to_datetime(train_without_id['date'], format='%m/%d/%Y')
#extract month, day and year into separate columns
train_without_id['SaleMonth']= train_without_id['date'].dt.month
train_without_id['SaleDat']= train_without_id['date'].dt.day
train_without_id['SaleYear']= train_without_id['date'].dt.year
#drop the original date column
train_without_id=train_without_id.drop(columns=['date'])
train_without_id.dtypes



bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
price            float64
SaleMonth          int32
SaleDat            int32
SaleYear           int32
dtype: object

In [50]:
test_without_id['date']=pd.to_datetime(test_without_id['date'], format='%m/%d/%Y')
#extract month, day and year into separate columns
test_without_id['SaleMonth']= test_without_id['date'].dt.month
test_without_id['SaleDat']= test_without_id['date'].dt.day
test_without_id['SaleYear']= test_without_id['date'].dt.year
#drop the original date column
test_without_id=test_without_id.drop(columns=['date'])
test_without_id.dtypes

bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
price            float64
SaleMonth          int32
SaleDat            int32
SaleYear           int32
dtype: object

In [51]:
#The feature yr_renovated is set to 0 if the house has not been renovated
train_without_id['age_since_renovated']=np.where(train_without_id['yr_renovated']!=0, 2024-train_without_id['yr_renovated'], 2024-train_without_id['yr_built'])
train_without_id
test_without_id['age_since_renovated']=np.where(test_without_id['yr_renovated']!=0, 2024-test_without_id['yr_renovated'], 2024-test_without_id['yr_built'])
test_without_id
#unique_bedrooms=sorted(train_without_id['age_since_renovated'].unique())
#unique_bedrooms

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,...,zipcode,lat,long,sqft_living15,sqft_lot15,price,SaleMonth,SaleDat,SaleYear,age_since_renovated
0,4,1.00,1520,9800,1.5,0,0,4,7,1520,...,98034,47.7303,-122.236,1540,7700,3.8900,8,6,2014,53
1,3,2.25,1800,15903,1.0,0,0,3,8,1340,...,98038,47.3813,-122.043,2000,15233,3.3562,3,16,2015,38
2,3,1.50,1780,5000,1.0,0,4,4,7,1030,...,98118,47.5168,-122.256,1780,7500,4.2000,12,12,2014,66
3,2,1.00,1240,6400,1.0,0,1,4,7,1060,...,98126,47.5493,-122.377,1240,6400,5.5900,6,18,2014,86
4,3,2.25,2190,8834,1.0,0,0,3,7,1390,...,98001,47.2633,-122.244,1490,8766,1.9425,10,30,2014,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,6,1.00,1370,5080,1.5,0,0,3,6,1120,...,98106,47.5238,-122.350,1020,5080,4.1500,1,20,2015,93
1996,3,1.75,1390,7399,1.0,0,0,4,7,1390,...,98031,47.4183,-122.182,1460,7800,2.4600,5,8,2014,49
1997,5,3.50,3700,7055,2.0,0,0,3,9,3700,...,98074,47.5929,-122.057,3170,6527,8.8995,2,11,2015,10
1998,3,2.50,1260,1102,3.0,0,0,3,8,1260,...,98107,47.6750,-122.387,1320,2500,4.4500,10,24,2014,17


In [52]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
y_train=train_without_id['price']
train_without_price=train_without_id.drop(columns=['price','yr_renovated','yr_built'])
scaler.fit(train_without_price)
scaler_train=scaler.transform(train_without_price)
len(scaler_train[0])

20

In [53]:
scaler=StandardScaler()
y_test=test_without_id['price']
test_without_price=test_without_id.drop(columns=['price','yr_renovated','yr_built'])
scaler.fit(test_without_price)
scaler_test=scaler.transform(test_without_price)
len(scaler_test[0])

20

In [60]:
#Part 1 (15 pts) Generate closed-form solution for reference.
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler


lr_model = SGDRegressor()
lr_model.fit(scaler_train, y_train)
y_pred = lr_model.predict(scaler_test)

print("Prediction on training set:", y_pred)
#print("Accuracy on training set:", lr_model.score(X, y))
print(y_test)
N = len(y_train)
rmse_train = np.sqrt(np.sum((np.array(y_test).flatten() - np.array(y_pred).flatten())**2)/N)
print(rmse_train)

Prediction on training set: [4.41766208 3.24385342 6.52331968 ... 8.5259631  4.38874174 4.69365841]
0       3.8900
1       3.3562
2       4.2000
3       5.5900
4       1.9425
         ...  
1995    4.1500
1996    2.4600
1997    8.8995
1998    4.4500
1999    3.7500
Name: price, Length: 2000, dtype: float64
1.0788850083816182


In [61]:
y_train_pred=lr_model.predict(scaler_train)
rmse_train = np.sqrt(np.sum((np.array(y_train).flatten() - np.array(y_train_pred).flatten())**2)/N)
rmse_train

1.9745252953752495

In [56]:
lr_model.coef_


array([-0.29240577,  0.28952719,  0.75207028,  0.11837506, -0.00737169,
        0.31578077,  0.44395958,  0.25587355,  1.02709899,  0.72542793,
        0.19040157, -0.25780791,  0.85163959, -0.33675653,  0.056313  ,
       -0.08257471,  0.0272527 , -0.07192868,  0.17046253,  0.6278845 ])

In [57]:
lr_model.intercept_

array([5.32634975])